In [1]:
import kaggle
import pandas as pd
import numpy as np
import os
import string
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, add
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm


In [2]:
import kaggle
import os

# Set Kaggle API key directory (assuming kaggle.json is in the same folder as this notebook)
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()

# Download the dataset
kaggle.api.dataset_download_files('adityajn105/flickr8k', path='flickr8k_data', unzip=True)

print("Download and unzip complete.")

Dataset URL: https://www.kaggle.com/datasets/adityajn105/flickr8k
Download and unzip complete.


In [11]:
CAPTIONS_FILE = 'flickr8k_data/captions.txt'
IMAGE_DIR = 'flickr8k_data/Images'

In [14]:
# Read captions file
def load_captions(filename):
    captions = []
    with open(filename, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if len(line) == 0:
                continue
            tokens = line.split('\t')
            if len(tokens) < 2:
                continue
            image_id = tokens[0].split('#')[0]
            caption = tokens[1]
            captions.append([image_id, caption])
    df = pd.DataFrame(captions, columns=['image', 'caption'])
    return df

captions_df = pd.read_csv(CAPTIONS_FILE)
captions_df.head()

,image,caption
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...
1,1000268201_693b08cb0e.jpg,A girl going into a wooden building .
2,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .
3,1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playh...
4,1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a woo...


In [13]:
with open(CAPTIONS_FILE, 'r', encoding='utf-8') as file:
    lines = file.readlines()
print(f"Total lines: {len(lines)}")
print("First 5 lines:")
for line in lines[:5]:
    print(line)

Total lines: 40456
First 5 lines:
image,caption

1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set of stairs in an entry way .

1000268201_693b08cb0e.jpg,A girl going into a wooden building .

1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .

1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playhouse .



In [15]:
def text_preprocessing(data):
    data['caption'] = data['caption'].apply(lambda x: x.lower())
    data['caption'] = data['caption'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
    data['caption'] = data['caption'].apply(lambda x: ' '.join([word for word in x.split() if len(word)>1]))
    data['caption'] = data['caption'].apply(lambda x: 'startseq ' + x + ' endseq')
    return data

captions_df = text_preprocessing(captions_df)
captions_df.head()

,image,caption
0,1000268201_693b08cb0e.jpg,startseq child in pink dress is climbing up se...
1,1000268201_693b08cb0e.jpg,startseq girl going into wooden building endseq
2,1000268201_693b08cb0e.jpg,startseq little girl climbing into wooden play...
3,1000268201_693b08cb0e.jpg,startseq little girl climbing the stairs to he...
4,1000268201_693b08cb0e.jpg,startseq little girl in pink dress going into ...


In [16]:
# Tokenizer: fit on captions
tokenizer = Tokenizer()
tokenizer.fit_on_texts(captions_df['caption'].values)

# Vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print("Vocabulary Size:", vocab_size)

# Maximum caption length
max_length = max(len(c.split()) for c in captions_df['caption'])
print("Max caption length:", max_length)


Vocabulary Size: 8811
Max caption length: 34


In [17]:
# Load InceptionV3 model
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input

inception_model = InceptionV3(weights='imagenet')
model_new = Model(inception_model.input, inception_model.layers[-2].output)

# Image preprocessing
def preprocess_image(img_path):
    img = load_img(img_path, target_size=(299, 299))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

# Feature extraction
def encode_image(img_path):
    img = preprocess_image(img_path)
    feature_vector = model_new.predict(img, verbose=0)
    feature_vector = np.reshape(feature_vector, feature_vector.shape[1])
    return feature_vector


In [18]:
feature_vector = encode_image(IMAGE_DIR + "/1000268201_693b08cb0e.jpg")
print(feature_vector.shape)

(2048,)


In [19]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, df, tokenizer, max_length, batch_size, image_path, vocab_size):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.batch_size = batch_size
        self.image_path = image_path
        self.vocab_size = vocab_size
        self.indexes = np.arange(len(self.df))
        
    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))
    
    def __getitem__(self, index):
        batch_indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        batch_df = self.df.iloc[batch_indexes]
        return self.__data_generation(batch_df)
    
    def __data_generation(self, batch_df):
        X1, X2, y = [], [], []
        
        for _, row in batch_df.iterrows():
            photo = encode_image(os.path.join(self.image_path, row['image']))
            seq = self.tokenizer.texts_to_sequences([row['caption']])[0]
            
            for i in range(1, len(seq)):
                in_seq, out_seq = seq[:i], seq[i]
                in_seq = pad_sequences([in_seq], maxlen=self.max_length)[0]
                out_seq = tf.keras.utils.to_categorical([out_seq], num_classes=self.vocab_size)[0]
                
                X1.append(photo)
                X2.append(in_seq)
                y.append(out_seq)
        
        return [np.array(X1), np.array(X2)], np.array(y)


In [20]:
# Define model architecture

# Image feature input
inputs1 = Input(shape=(2048,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)

# Sequence input (captions)
inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)

# Combine
decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)

# Define full model
model = Model(inputs=[inputs1, inputs2], outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Summary
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)    │ (None, 34)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_1 (InputLayer)    │ (None, 2048)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding (Embedding)         │ (None, 34, 256)           │       2,255,616 │ input_layer_2[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout (Dropout)             │ (None, 2048)              │               0 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_1 (Dropout)           │ (None, 34, 256)           │               0 │ embedding[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ not_equal (NotEqual)          │ (None, 34)                │               0 │ input_layer_2[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 256)               │         524,544 │ dropout[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm (LSTM)                   │ (None, 256)               │         525,312 │ dropout_1[0][0],           │
│                               │                           │                 │ not_equal[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add (Add)                     │ (None, 256)               │               0 │ dense[0][0], lstm[0][0]    │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_1 (Dense)               │ (None, 256)               │          65,792 │ add[0][0]                  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_2 (Dense)               │ (None, 8811)              │       2,264,427 │ dense_1[0][0]              │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 5,635,691 (21.50 MB)

 Trainable params: 5,635,691 (21.50 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
from tqdm.auto import tqdm

# Precompute image features for all unique images
image_features = {}

unique_images = captions_df['image'].unique()

for img_name in tqdm(unique_images):
    img_path = os.path.join(IMAGE_DIR, img_name)
    image_features[img_name] = encode_image(img_path)

print(f"Extracted features for {len(image_features)} images.")


  0%|          | 0/8091 [00:00<?, ?it/s]

Extracted features for 8091 images.


In [24]:
import pickle

with open('image_features.pkl', 'wb') as f:
    pickle.dump(image_features, f)

print("Saved image_features.pkl")

Saved image_features.pkl


In [26]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, df, tokenizer, max_length, batch_size, image_features, vocab_size, shuffle=True):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.batch_size = batch_size
        self.image_features = image_features
        self.vocab_size = vocab_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.df))
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))

    def __getitem__(self, index):
        batch_indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        batch_df = self.df.iloc[batch_indexes]
        return self.__data_generation(batch_df)

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, batch_df):
        X1, X2, y = [], [], []

        for _, row in batch_df.iterrows():
            img_feature = self.image_features[row['image']]
            seq = self.tokenizer.texts_to_sequences([row['caption']])[0]

            for i in range(1, len(seq)):
                in_seq, out_seq = seq[:i], seq[i]
                in_seq = pad_sequences([in_seq], maxlen=self.max_length)[0]
                out_seq = tf.keras.utils.to_categorical([out_seq], num_classes=self.vocab_size)[0]

                X1.append(img_feature)
                X2.append(in_seq)
                y.append(out_seq)

        return [np.array(X1), np.array(X2)], np.array(y)


In [27]:
# Create generator
batch_size = 64

train_generator = DataGenerator(
    captions_df, tokenizer, max_length, batch_size, image_features, vocab_size
)


In [28]:
import pickle

# Save image_features
with open('image_features.pkl', 'wb') as f:
    pickle.dump(image_features, f)

print("Saved image_features.pkl ")


Saved image_features.pkl 


In [ ]:
import pickle

# Load image_features
with open('image_features.pkl', 'rb') as f:
    image_features = pickle.load(f)

print(f"Loaded {len(image_features)} image features ")

In [ ]:
# Train model
epochs = 10
steps_per_epoch = len(train_generator)

model.fit(train_generator, epochs=epochs, steps_per_epoch=steps_per_epoch)